A time resolved spectroscopy estimator
=======================

Do a time resolved spectral fit for the PKS 2155-304 flare on 15 minutes time intervals.

Prerequisites
-------------

-  Understanding of how the light curve estimator works, please refer to
   :doc:`light curve notebook </tutorials/analysis-time/light_curve>`, and :doc:`light curve for flares notebook </tutorials/analysis-time/light_curve_for_flares>`.

Context
-------

In the analysis of flaring sources, it is often interesting to study not only how
the flux changes with time but also the variability of the spectral shape and hardness.

A typical example is given by the flare of PKS 2155-304 during the night
from July 29 to 30 2006. See the `following
article <https://ui.adsabs.harvard.edu/abs/2009A%26A...502..749A/abstract>`__.

**Objective: Model the varibility of the spectral index of 
PKS 2155-304 in 15-minutes time intervals.**

Proposed approach
-----------------

We have seen in the general presentation of the light curve estimator,
see the :doc:`light curve notebook </tutorials/analysis-time/light_curve>`, and how to select
and produce datasets on smaller time steps in `light curve notebook </tutorials/analysis-time/light_curve_for_flares>`.

After the selection, we can initially model the spectral shape as a Power Law,
and loop the model fitting over the produced datasets to obtain the evolution of the spectral index.
We can also test different models to see which one best tracks the observations.

In summary, we have to:

-  Select relevant `~gammapy.data.Observations` from the
   `~gammapy.data.DataStore`
-  Apply the time selection in our predefined time intervals to obtain a
   new `~gammapy.data.Observations`
-  Perform the data reduction (in 1D or 3D)
-  Define the initial source model
-  Fit the model in each interval and extract the spectral index

Here, we will use the PKS 2155-304 observations from the
`H.E.S.S. first public test data release <https://www.mpi-hd.mpg.de/hfm/HESS/pages/dl3-dr1/>`__.
We will use time intervals of 15 minutes
duration. The tutorial is implemented with the intermediate level API.

Setup
-----

As usual, we’ll start with some general imports…



In [ ]:
import logging
import numpy as np
import astropy.units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.time import Time
from astropy.table import QTable
from regions import CircleSkyRegion

# %matplotlib inline
import matplotlib.pyplot as plt

log = logging.getLogger(__name__)

from gammapy.data import DataStore, GTI
from gammapy.datasets import Datasets, SpectrumDataset

from gammapy.estimators import LightCurveEstimator, Estimator
from gammapy.estimators.utils import get_rebinned_axis
from gammapy.makers import (
    ReflectedRegionsBackgroundMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
)
from gammapy.maps import MapAxis, RegionGeom, TimeMapAxis
from gammapy.modeling import Fit
from gammapy.modeling.models import PowerLawSpectralModel, SkyModel, BrokenPowerLawSpectralModel, Models, SpectralModel

In [ ]:
log = logging.getLogger(__name__)

### Data selection

We select the datastore and restrict it to events within 2 degrees of PKS2155-304.

In [ ]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1/")
target_position = SkyCoord(329.71693826 * u.deg, -30.2255890 * u.deg, frame="icrs")
selection = dict(
    type="sky_circle",
    frame="icrs",
    lon=target_position.ra,
    lat=target_position.dec,
    radius=2 * u.deg,
)
obs_ids = data_store.obs_table.select_observations(selection)["OBS_ID"]
observations = data_store.get_observations(obs_ids)
print(f"Number of selected observations : {len(observations)}")

We define 10-minutes time intervals as lists of ``~astropy.Time`` start and stop objects, and apply the intervals to the observations by using ``~observations.select_time``

In [ ]:
t0 = Time("2006-07-29T20:30")
duration = 15 * u.min
n_time_bins = 25
times = t0 + np.arange(n_time_bins) * duration
time_intervals = [Time([tstart, tstop]) for tstart, tstop in zip(times[:-1], times[1:])]
print(time_intervals[-1].mjd)
short_observations = observations.select_time(time_intervals)
# check that observations have been filtered
print(f"Number of observations after time filtering: {len(short_observations)}\n")
print(short_observations[1].gti)

## Data reduction

We define the energy axes. As usual, the true energy axis has to cover a
wider range to ensure a good coverage of the measured energy range
chosen.

We need to define the ON extraction region. Its size follows typical
spectral extraction regions for H.E.S.S. analyses.

In [ ]:
# Target definition
energy_axis = MapAxis.from_energy_bounds("0.4 TeV", "20 TeV", nbin=10)
energy_axis_true = MapAxis.from_energy_bounds(
    "0.1 TeV", "40 TeV", nbin=20, name="energy_true"
)

on_region_radius = Angle("0.11 deg")
on_region = CircleSkyRegion(center=target_position, radius=on_region_radius)

geom = RegionGeom.create(region=on_region, axes=[energy_axis])

We now create the dataset and background makers for the selected
geometry and to the actual data reduction

In [ ]:
dataset_maker = SpectrumDatasetMaker(
    containment_correction=True, selection=["counts", "exposure", "edisp"]
)
bkg_maker = ReflectedRegionsBackgroundMaker()
safe_mask_masker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

In [ ]:
datasets = Datasets()

dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true)

for obs in short_observations:
    dataset = dataset_maker.run(dataset_empty.copy(), obs)

    dataset_on_off = bkg_maker.run(dataset, obs)
    dataset_on_off = safe_mask_masker.run(dataset_on_off, obs)
    datasets.append(dataset_on_off)

## Modeling

We set the initial model: he power law spectral model of index 3.4 used in the
`reference
paper <https://ui.adsabs.harvard.edu/abs/2009A%26A...502..749A/abstract>`__.

This model will be re-fitted in each time interval to investigate the change in spectral hardness.

In [ ]:
spectral_model = PowerLawSpectralModel(
    index=3.4, amplitude=2e-11 * u.Unit("1 / (cm2 s TeV)"), reference=1 * u.TeV
)
spectral_model.parameters["index"].frozen = False

sky_model = SkyModel(spatial_model=None, spectral_model=spectral_model, name="pks2155")

## Time resolved spectroscopy algorithm

Here we define the algorithm used to fit the sky model in each temporal bin..

In [ ]:
def time_resolved_spectroscopy(datasets, model, time_intervals):
    fit = Fit()
    valid_intervals = []
    fit_results = []
    index = 0
    for t_min, t_max in time_intervals:
        datasets_to_fit = datasets.select_time(
            time_min=t_min, time_max=t_max
        )

        if len(datasets_to_fit) == 0:
            log.info(
                f"No Dataset for the time interval {t_min} to {t_max}. Skipping interval."
            )
            continue

        model_in_bin = model.copy(name="Model_bin_" + str(index))
        datasets_to_fit.models = model_in_bin
        result = fit.run(datasets_to_fit)
        fit_results.append(result)
        valid_intervals.append([t_min, t_max])
        index += 1

    return valid_intervals, fit_results

... and apply it to our data.

In [ ]:
times, results = time_resolved_spectroscopy(datasets, sky_model, time_intervals)

To better visualize the data, we can create a table with the fit results.

In [ ]:
def create_table(time_intervals, fit_result):

    t = QTable()

    t["tstart"] = np.array(times).T[0]
    t["tstop"] = np.array(times).T[1]
    t["convergence"] = [result.success for result in fit_result]
    for par in fit_result[0].models.parameters.free_parameters:
        t[par.name] = [result.models.parameters[par.name].value*par.unit for result in fit_result]
        t[par.name+"_err"] = [result.models.parameters[par.name].error*par.unit for result in fit_result]
    
    return t

In [ ]:
table = create_table(times, results)

... and plot the spectral index as a function of amplitude, highlighting the evolution in time to create a hysteresis plot.

In [ ]:
amp = table["amplitude"]
indexes = table["index"]
times = table["tstart"]

In [ ]:
plt.scatter(amp, indexes, c=times.mjd)
plt.plot(amp, indexes, linewidth=0.5)
plt.show()

## Exercises

Rerun the algorithm using a different spectral shape, such as a broken power law. Compare the significance of the new model with the simple power law. Take note of any fit non-convergence in the bins. 